In [16]:
# modules

import numpy as np
import matplotlib.pyplot as plt
import sys
import os

sys.path.append(os.pardir)

from dataset.mnist import load_mnist
from common.math_functions import *

%load_ext rpy2.ipython

# 5章 誤差逆伝搬法

> [p123] なお、誤差逆伝播法を計算グラフによって説明するアイデアは、Andrej Karpathy のブログ [4] 、また、彼と Fei-Fei Li 教授らによって行われたスタンフォード大学の ディープラーニングの授業「CS231n」[5] を参考にしています。

[Andrej Karpathy](https://cs.stanford.edu/people/karpathy/) さんは Tesla の CV 系 AI 研究者で，ブログは[ここ](http://karpathy.github.io/neuralnets/)．PhD の指導教官が Stanford の [Fei-Fei Li](https://profiles.stanford.edu/fei-fei-li?tab=bio) さんで，この方は GoogleCloud で Chief Scientist してたらしい．

# 5.1. 計算グラフ

> [p123] 計算グラフとは、計算の過程をグラフによって表したものです。ここで言うグラフ とは、データ構造としてのグラフであり、複数のノードとエッジによって表現されます
 
計算グラフは「演算のフローを可視化したグラフ」という感じ．

> [p124] 計算グラフはノードと矢印によって計算の過程を表します。ノードは○で表記し、 ○の中に演算の内容を書きます。また、計算の途中結果を矢印の上部に書くことで、 ノードごとの計算結果が左から右へ伝わるように表します。

前述のニューロンのグラフ (図3-18とか) ではノードが「変数(入力,特徴量,出力)」で矢印が「変換(写像)」というイメージだったが，ここの計算グラフ(図5-3とか)では逆でノードが「演算(変換,写像)」で矢印が「変数(入力,特徴量,出力)」というイメージ．混同しないように．

> [p126] ここまで見てきたように、計算グラフを使って問題を解くには、
1. 計算グラフを構築する
2. 計算グラフ上で計算を左から右へ進める
>
>という流れで行います。ここで 2 番目の「計算を左から右へ進める」というステップは、順方向の伝播、略して、**順伝播**(forward propagation)と言います。順伝播は、計算グラフの出発点から終着点への伝播です。順伝播という名前があるのであれば、逆方向――図で言うと、右から左方向へ――の伝播も考えることができるでしょう。実際、それを**逆伝播**(backward propagation)と言います。逆伝播は、この先、 微分を計算するにあたって重要な働きをします。

そもそも計算グラフは「演算の過程を左スタート右ゴールで可視化したもの」なので，左から右へ計算を進める過程を「順伝搬」と呼ぶのは自然．

> [p127] たとえば、リンゴとそれ以外の買い物を合計する計算―― 4,000 + 200 → 4,200――は、4,000 という数字がどのように計算されてきたかということについては考えずに、ただ 2 つの数字を足せばよいということを意味します。言い換えれば、各ノードの計算で行うべきことは、自分に関係する計算――この例では、入力された 2 つの数字の足し算――だけであり、 全体のことについては何も考えなくてよいのです。
このように、計算グラフでは、局所的な計算に集中することができます。たとえ全体の計算がどんなに複雑であったとしても、各ステップでやることは、対象とするノードの「局所的な計算」なのです。局所的な計算は単純ですが、その結果を伝達することで、全体を構成する複雑な計算の結果が得られます。... 計算グラフも、複雑な計算を「単純で局所的な計算」に分割して、流れ 作業を行うように、計算の結果を次のノードへと伝達していきます

計算グラフのこのメリットは，関数型プログラミングパラダイムのメリットと似ている．大きい処理を細かい関数に分けることで，各関数の入出力の仕様だけに注意を払って実装とか単体テストができるようになり，気持ちが楽でミスりづらいしデバッグしやすい．

> [p128] ここでは、リンゴの値段に関する微分だけを求めましたが、「消費税に関する支払金額の微分」や「リンゴの個数に関する支払金額の微分」も同様の手順で求めることができます。そして、その際には、途中まで求めた微分(途中まで流れた微分)の結果を共有することができ、効率良く複数の微分を計算することができるのです。

ここが back propagation の肝だと思う．途中までの微分の結果を共有できるので，無駄が削られ計算コストが下がる．

電気信号回路(ニューロン)でふわっとしたイメージすると...  
これまでのナイーブな勾配算出法は，
- 第1層の各ニューロンから出る信号をちょっと強めて，最後の信号 (出力層から出た信号をロス関数に入れて出る信号) がどのくらい変化するか見る．それで第1層の偏微分係数ゲット．
- 第2層の各ニューロンから出る信号をちょっと強めて，最後の信号がどのくらい変化するか見る．それで第2層の偏微分係数ゲット．
- ...

という感じ．ここでポイントなのが，例えば第 $n-1$ 中間層と第 $n$層(出力層)の間には，何度も(全パラメータ数分)だけ信号が流れている．  
一方，誤差逆伝搬では，

- 第 $n-1$ 層の各ニューロンから出る信号をちょっと強めて，最後の信号がどのくらい変化するか見る．それで第 $n-1$ 層の偏微分係数ゲット．そしてその情報を記録してとっておく．
- 第 $n-2$ 層の各ニューロンから出る信号をちょっと強めて，次の第 $n-1$ 層に入る信号がどのくらい変化するかを見る．それを先ほど記録した $n-1$ 層の偏微分係数に掛け合わせて，第 $n-2$ 層の変微分係数を取得．この情報も記録しておく．
- ...

という感じ．連鎖律を使ったことで，例えば先ほどの方法だと何度も信号が流されていた $n-1$ 層と $n$ 層の間に，１回だけしか信号が流れていない．無駄を削ぎ落とせている．

# 5.2. 連鎖律

計算グラフのノードは

- 左から入力された信号に対しては，その値をノード関数で変換して結果を右に出力する
- 右から入力された信号に対しては，それにノード関数の微分を掛け合わせて左に出力する

という挙動を示す(ことにしている)．そうすると色々便利だから．

> [p131] 図 5-7 で注目すべきは、一番左の逆伝播の結果です。これは、連鎖律より、
...「x に関する z の微分」に対応します。つまり、逆伝播が行っていることは、連鎖律の原理から構成されているのです。

もはや計算グラフは「合成関数の微分を分かりやすく求めるためのツール」とも捉えられそう．普通に微積のテストとかでも役立ちそう．

少し違っていて注意しないといけないのは， SGD においては「合成関数の導関数」ではなく 「合成関数の微分係数」という１点の値を求めようとしてる，ということ．

# 5.3. 逆伝搬

> [p132] なお、この例では上流から伝わった微分の値を $\frac{\partial L}{\partial z}$ としましたが、これは図5-10に示すように、最終的に $L$ という値を出力する大きな計算グラフを想定しているため

NN の SGD とかでは「各パラメータをちょっと動かした時にロス $L$ がどのくらい変化するか」という勾配を求めようとするので，計算グラフの最後には必ず，ロス $L$ を出力するロス関数ノードがあるはず．

# 5.4. 単純なレイヤの実装

> 次節では、ニューラルネットワークを構成する「層(レイヤ)」をひとつのクラスで実装することにします。ここで言う「レイヤ」とは、ニューラルネットワー クにおける機能の単位です。たとえば、シグモイド関数のための Sigmoid や、 行列の積のための Affine など、レイヤ単位で実装を行います。そのため、こ こでも「レイヤ」という単位で、乗算ノードと加算ノードを実装します。

次節以降 (例えば図5-28) では，当初の俺が思ってた通り変換 (変数じゃなくて) を層 (レイヤー) とみなしている．このグラフでは変換がノードで表されててノード群を層と見るのは視覚的に直感的．この捉え方なら $n$ 層ニューラルネットワークっていう用語も「$n$ 回(層)の変換を行う NN」と解釈できて，分かりやすい．

そう考えると，前述の図3-18みたいな「変数がノードで変換が矢印」のグラフより，後述の図5-14~5-30みたいな「変数が矢印で変換がノード」のグラフの方が「層(レイヤー)」のイメージがつきやすくて良いかも．まあ，どっちの見方もできるようにしとくのが良い．

計算グラフは NN を実装する上での良い設計図になっているってことか．ノード(あるいは複数ノードをまとめたレイヤー)を class で実装して，そのインスタンスを組み合わせることで NN 全体を実装できるから．Pytorch もそんな感じだった．

In [3]:
# 乗算レイヤの実装 (5.4.1 / p137)

class MulLayer:
    """掛け算レイヤを表すクラス
    
    f(x,y) = xy という変換に対応するクラス．
    計算グラフのノードをそのまま実装するイメージ．
    
    Attributes:
      x: 掛け算関数f(x,y) = x*y の x．
      y: 同じく． のちの NN では，更新が繰り返されるパラメータ値が状態として保存されている．
    """
    
    def __init__(self):
        """ initialize
        
        アトリビュートの初期化を __init__() ではなく forward() メソッドでやる理由
        ニューラルネットで逆伝搬する時のフローをイメージしてる．
        １回だけ予測値とロスを算出して(forward して)，
        そこから戻りながら勾配を計算していく(backward する)というフローが採られるので，
        それを想定して最初の forward 時にアトリビュートの初期化が起こるように書いてある．
        
        """
        self.x = None
        self.y = None
        
    def forward(self, x, y):
        """掛け算レイヤの forward 処理
        
        つまり，ただ掛け算を実行して結果を出力する．
        ここでアトリビュート(状態)の初期化や更新が行われる理由は，↑を参照．
        
        Returns:
          - self.x * self.y
        
        """
        self.x = x
        self.y = y
        return x * y
    
    def backward(self, dout):
        """掛け算レイヤの backward 処理
        
        つまり，状態 x,y に保存されている値での掛け算関数の勾配を計算する．
        つまり，座標(self.x, self.y)における掛け算関数の勾配を計算する．
        さらに(計算グラフ上の右から)入力された値 dout に算出された勾配を掛け合わせる．
        そしてそれを出力する．
        
        Returns:
          - self.x における f(x,y)=x*y の偏微分係数(勾配)
          - self.y における f(x,y)=x*y の偏微分係数(勾配)        
        
        """
        dx = dout * self.y
        dy = dout * self.x
        return dx, dy

In [7]:

# get layer

mul_apple_layer = MulLayer()
mul_tax_layer = MulLayer()
# 各レイヤ(計算グラフのノード)ごとのオブジェクトを作成．


# forward

apple_price = 100
apple_num = 2
tax = 1.1

apple_sum = mul_apple_layer.forward(x=apple_price, y=apple_num)
print(apple_sum)  # forward 伝搬での出力
print(mul_apple_layer.x, mul_apple_layer.y)  # 状態に forward 伝搬での入力が記録されている

total_pay = mul_tax_layer.forward(x=apple_sum, y=tax)
print(total_pay)  # forward 伝搬での出力
print(mul_tax_layer.x, mul_tax_layer.y)  # 状態に forward 伝搬での入力が記録されてる


# backward

dtotal_pay = 1
# いまの合成関数全体の最終出力は total_pay で，
# backward に　chain rule で勾配を求めていく都合上，
# 最初に d(total_pay) / d(total_pay) という自分自身による微分係数(常に１)を作っとく必要がある．

dapple_sum, dtax = mul_tax_layer.backward(dtotal_pay)
print(dapple_sum, dtax)
# apple_sum, tax の(forward 時に属性に記録した値での) (total_pay に対する) 偏微分係数を算出．
# これはノードから左方向(逆方向)に吐き出されるイメージ．
# d(total_pay) / d(apple_sum)  と  d(total_pay) / d(tax)　という感じ．
# 最終出力(目的関数値) total_pay に直接影響してる変数なので， chain rule は活用してない．

dapple_price, dapple_num = mul_apple_layer.backward(dapple_sum)
print(dapple_price, dapple_num)
# apple_price, apple_num の (属性に記録された値での) (total_payに対する) 変微分係数を取得．
# これはノードから左方向(逆方向)に吐き出されるイメージ．
# ここで，実際に中身で計算されているのは
# d(apple_sum) / d(apple_price)  と  d(apple_sum) / d(apple_num)  だけで，
# この２つに右からの入力で渡した dapple_sum == d(total_pay) / d(apple_sum) を掛けて，
# d(apple_sum) / d(apple_price) × d(total_pay) / d(apple_sum)
#  = d(apple_price) / d(total_pay) という感じで．
# 連鎖律(合成関数微分，chain rule) を活用して楽に算出している．

200
100 2
220.00000000000003
200 1.1
1.1 200
2.2 110.00000000000001


In [9]:
class AddLayer:
    """足し算レイヤ
    
    f(x, y) = x + y の変換に対応するレイヤ
    
    Attributes:
      x: 足し算レイヤに左から入力された x つまり f(x,y) = x + y の x．
      y: 同じく． forward() では足し算に使われ， backward() では勾配を求めたい座標となる．
          NN では繰り返し更新され続けるパラ値が状態として記録される．
    
    """
    
    def __init__(self):
        """ initialize
        
        初期化．
        
        """
        self.x = None
        self.y = None
        
    def forward(self, x, y):
        """ forward propagation
        
        足し算を実行して右方向に出力．
        
        """
        self.x = x
        self.y = y
        return x + y
    
    def backward(self, dout):
        """ backward propagation
        
        アトリビュートに保存された座標での足し算関数に対する勾配を算出し，
        それを dout に chain rule で掛け合わせ， 左へ出力．
        
        Note:
          掛け算レイヤの時と違って，偏微分係数は座標によらず一定である．
          なので， self.x と self.y に forward 時の入力を記録しておく必要も，今回は無い．
          ただまあ，一般的には座標によって勾配違うので，ちゃんと self.x,y に記録することにする．
        
        """
        dx = 1 * dout  # d(final) / d(x)  =  d(out) / d(x) * d(final) / d(out)
        dy = 1 * dout  # d(final) / d(y)  =  d(out) / d(y) * d(final) / d(out)
        return dx, dy

In [14]:

# makeup Layers

mul_apple_layer = MulLayer()
mul_orange_layer = MulLayer()
add_fruit_layer = AddLayer()
mul_tax_layer = MulLayer()


# forward propagation

apple_num = 2
apple_price = 100
orange_num = 3
orange_price = 150
tax = 1.1

apple_sum = mul_apple_layer.forward(x=apple_num, y=apple_price)
orange_sum = mul_orange_layer.forward(x=orange_num, y=orange_price)
fruit_sum = add_fruit_layer.forward(x=apple_sum, y=orange_sum)
total_pay = mul_tax_layer.forward(x=fruit_sum, y=tax)
print(apple_sum, orange_sum)
print(fruit_sum)
print(total_pay)


# backward propagation

d_total_pay = 1    #  d(total_pay) / d(total_pay)

d_fruit_sum, d_tax = mul_tax_layer.backward(dout=d_total_pay)
print( d_fruit_sum, d_tax)
#  d(total_pay) / d(fruit_sum)  と  d(total_pay) / d(tax)

d_apple_sum, d_orange_sum = add_fruit_layer.backward(dout=d_fruit_sum)
print( d_apple_sum, d_orange_sum )
#  d(total_pay) / d(apple_sum)
#    =  d(total_pay) / d(fruit_sum)  *  d(fruit_sum) / d(apple_sum)
#  d(total_pay) / d(orange_sum)
#    = d(total_pay) / d(fruit_sum)  * d(fruit_sum) / d(orange_sum)

d_apple_num, d_apple_price = mul_apple_layer.backward(dout=d_apple_sum)
print( d_apple_num, d_apple_price )
#  d(total_pay) / d(apple_num)
#    = d(total_pay) / d(apple_sum)  *  d(apple_sum) / d(apple_num)
# d(total_pay) / d(apple_price)
#    = d(total_pay) / d(apple_sum) * d(apple_sum) / d(apple_price)

d_orange_num, d_orange_price = mul_orange_layer.backward(dout=d_orange_sum)
print( d_orange_num, d_orange_price )
#  d(total_pay) / d(orange_num)
#    = d(total_pay) / d(orange_sum)  *  d(orange_sum) / d(orange_num)
#  d(total_pay) / d(orange_price)
#    = d(total_pay) / d(orange_sum)  *  d(orange_sum) / d(orange_price)

200 450
650
715.0000000000001
1.1 650
1.1 1.1
110.00000000000001 2.2
165.0 3.3000000000000003


> 図 5-17

計算グラフでは変数が矢印で表現されているが，

- forward 矢印にはその変数の値
- backward 矢印にはその変数の値における最終出力の勾配(偏微分係数)

が対応している．そう捉えると分かりやすい．

# 5.5. 活性化関数レイヤの実装

> それでは、計算グラフの考え方をニューラルネットワークに適用したいと思います。ここでは、ニューラルネットワークを構成する「層(レイヤ)」をひとつのクラスとして実装することにします。

ここでの層(レイヤ)は NN の変換(アフィン, 活性化など)のことを指してる．最初の方(図 2-13, 図3-1とか)ではニューロン群を層と見做していたが，今は違う．まあどっちの見方もメリットがある (前者は back/forward propagation の実装がしやすく，後者は神経ネットワーク的な感じで分かりやすい)ので，どっちもできるように．

## 5.5.1.  ReLU レイヤ

In [33]:
# ReLU 活性化関数クラスの実装

class ReLU:
    """ ReUL layer
    
    テキストの実装から自分好みに変えてる
    
    """
    
    def __init__(self):
        """ initialize
        """
        self.x = None
    
    def forward(self, x):
        """ forward propagation
        
        activate vector x by ReLU function.
        
        """
        self.x = x
        out = np.where(x >= 0, x, 0)
        return out
    
    def backward(self, dout):
        """ backward propagation
        
        calcurate gradient by using chain rule.
        d(loss) / d(x)  =  d(loss) / d(out)  *  d(out) / d(x)
        
        """
        dx = np.where(self.x >= 0, 1, 0)
        return dout * dx

In [34]:
# ↑の ReLU クラス実装の試行錯誤

x = np.array([0.9, -0.4, 0.5, -0.3])
x >= 0
print( np.where(x >= 0, x, 0) )
# https://note.nkmk.me/python-numpy-where/

%R x <- c(0.9, - 0.4, 0.5, - 0.3)
%R print( ifelse(x >= 0, x, 0) )


print( np.where(x >= 0, 1, 0) )
%R print( ifelse(x >=0, 1, 0))

[0.9 0.  0.5 0. ]


[1] 0.9 0.0 0.5 0.0


[1 0 1 0]


[1] 1 0 1 0


## 5.5.2.  Sigmoid レイヤ

$$
\begin{aligned}
\frac{dy}{dx}
&= \frac{d}{dx} \left[ \frac{1}{1 + \exp(-x)} \right] \\
&= -1 * \left(1 + \exp(-x) \right)^{-2}
 \frac{d}{dx} [1+\exp(-x)]  \\
&= \left(1 + \exp(-x) \right)^{-2} \exp(-x)  \\
&= \frac{1}{1 + \exp(-x)} \frac{\exp(-x)}{1 + \exp(-x)} \\
&= \frac{1}{1 + \exp(-x)} \left( 1 - \frac{1}{1 + \exp(-x)} \right) \\
&= y~(1-y)
\end{aligned}
$$

In [35]:
# Sigmoid レイヤの実装

class Sigmoid:
    """ sigmoid layer
    
    テキストの実装から自分好みに少し変えてる．
    
    """
    
    def __init__(self):
        """ initialize 
        """
        x = None
    
    def forward(self, x):
        """ forward propagation
        
        勾配 d(out) / d(x) を計算する際，↑で確認したように，
        シグモイドの入力 x を使うより出力 y を使った方が簡単に済む．
        なので， self.x = x として x を記録しておくのではなく，　out　の方を記録しておいて，
        それを backward() の時に使うようにする．
        
        """
        out = 1 / (1 + np.exp(-x))
        self.out = out
        return out
    
    def backward(self, dout):
        """ backward propagation 
        
        ↑で確認したように解析的に勾配 d(out) / d(x) が求まる．
        chain rule を使って
        d(loss) / d(x)  =  d(loss) / d(out)  *  d(out) / d(x)
        を算出して返す．
        
        """
        dx = self.out * (1.0 - self.out)
        return dout * dx